In [21]:
pip install vaderSentiment pandas

  Obtaining dependency information for vaderSentiment from https://files.pythonhosted.org/packages/76/fc/310e16254683c1ed35eeb97386986d6c00bc29df17ce280aed64d55537e9/vaderSentiment-3.3.2-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/126.0 kB ? eta -:--:--
   --- ------------------------------------ 10.2/126.0 kB ? eta -:--:--
   --- ------------------------------------ 10.2/126.0 kB ? eta -:--:--
   --------- ----------------------------- 30.7/126.0 kB 325.1 kB/s eta 0:00:01
   ------------ -------------------------- 41.0/126.0 kB 279.3 kB/s eta 0:00:01
   ---------------------- ---------------- 71.7/126.0 kB 393.8 kB/s eta 0:00:01
   -------------------------------------- 126.0/126.0 kB 570.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from textblob import TextBlob

import praw

import tweepy

from tweepy.errors import Forbidden, TweepyException
 
# Reddit Scraper Function with Sentiment Analysis

def reddit_scraper(subreddit_name, limit=10):

    reddit = praw.Reddit(

        client_id='XXUl08uRXYb_rAaQi_M2mQ',  # Your client ID

        client_secret='-ecLwM5dQGXnyE7dZovAvrmKrywCzg',  # Your client secret

        user_agent='test-api'  # Your user agent

    )

    subreddit = reddit.subreddit(subreddit_name)

    top_posts = subreddit.top(limit=limit)

    posts = []

    for post in top_posts:

        sentiment = TextBlob(post.title).sentiment

        posts.append({

            'platform': 'Reddit',

            'title': post.title,

            'score': post.score,

            'url': post.url,

            'num_comments': post.num_comments,

            'created_utc': post.created_utc,

            'author': str(post.author),

            'sentiment': sentiment.polarity,  # Sentiment polarity (range: -1 to 1)

            'subjectivity': sentiment.subjectivity  # Sentiment subjectivity (range: 0 to 1)

        })

    return posts
 
# Twitter Scraper Function with Sentiment Analysis

def twitter_scraper(bearer_token, search_query=None, username=None, tweet_count=10):

    client = tweepy.Client(bearer_token=bearer_token)

    tweets = []

    try:

        if search_query:

            response = client.search_recent_tweets(query=search_query, max_results=tweet_count)

            for tweet in response.data:

                sentiment = TextBlob(tweet.text).sentiment

                tweets.append({

                    'platform': 'Twitter',

                    'username': tweet.author_id,

                    'text': tweet.text,

                    'created_at': tweet.created_at,

                    'retweets': tweet.public_metrics['retweet_count'],

                    'favorites': tweet.public_metrics['like_count'],

                    'sentiment': sentiment.polarity,  # Sentiment polarity (range: -1 to 1)

                    'subjectivity': sentiment.subjectivity  # Sentiment subjectivity (range: 0 to 1)

                })

        elif username:

            user = client.get_user(username=username)

            response = client.get_users_tweets(user.id, max_results=tweet_count)

            for tweet in response.data:

                sentiment = TextBlob(tweet.text).sentiment

                tweets.append({

                    'platform': 'Twitter',

                    'username': username,

                    'text': tweet.text,

                    'created_at': tweet.created_at,

                    'retweets': tweet.public_metrics['retweet_count'],

                    'favorites': tweet.public_metrics['like_count'],

                    'sentiment': sentiment.polarity,  # Sentiment polarity (range: -1 to 1)

                    'subjectivity': sentiment.subjectivity  # Sentiment subjectivity (range: 0 to 1)

                })

        else:

            print("Please provide a search query or username.")

    except Forbidden as e:

        print(f"Twitter API Forbidden Error: {e}")

    except TweepyException as e:

        print(f"Twitter API Error: {e}")

    return tweets
 
# Combined Function to choose between Reddit and Twitter

def combined_scraper(platform, subreddit_name=None, search_query=None, username=None, limit=10, tweet_count=10):

    if platform.lower() == 'reddit' and subreddit_name:

        return reddit_scraper(subreddit_name, limit=limit)

    elif platform.lower() == 'twitter':

        return twitter_scraper(

            bearer_token="AAAAAAAAAAAAAAAAAAAAANOSvgEAAAAA67OERHp4by%2Fk3Vk1surAzO8rDcw%3DgpZO4AySy5QViApQcrD4k1pXleEIukKucwK0PajDXJCW4jyHop",

            search_query=search_query,

            username=username,

            tweet_count=tweet_count

        )

    else:

        print("Invalid platform or missing necessary arguments.")

        return []
 
# Example usage

platform_choice = input("Choose platform (Reddit/Twitter): ").strip().lower()
 
posts = []  # Initialize posts variable
 
if platform_choice == 'reddit':

    subreddit_name = input("[+] What subreddit do you want: ").strip()

    posts = combined_scraper(platform='reddit', subreddit_name=subreddit_name, limit=50)

elif platform_choice == 'twitter':

    search_query = input("[+] Enter Twitter search query (leave blank if using username): ").strip()

    username = input("[+] Enter Twitter username (leave blank if using search query): ").strip() or None

    posts = combined_scraper(platform='twitter', search_query=search_query, username=username, tweet_count=50)

else:

    print("Invalid platform choice. Please enter 'Reddit' or 'Twitter'.")
 
# Print results

for post in posts:

    if post['platform'] == 'Reddit':

        print(f"[Reddit] {post['title']} by {post['author']} (Score: {post['score']}, Comments: {post['num_comments']}) - {post['url']}")

        print(f"Sentiment Polarity: {post['sentiment']}, Subjectivity: {post['subjectivity']}")

    elif post['platform'] == 'Twitter':

        print(f"[Twitter] @{post['username']} tweeted: {post['text']} on {post['created_at']} (Retweets: {post['retweets']}, Favorites: {post['favorites']})")

        print(f"Sentiment Polarity: {post['sentiment']}, Subjectivity: {post['subjectivity']}")
 
        
 